# How to find a YouTube channel Id?

One way to get a channel Id is to go through the source code of youtube channel page and look for Id. [You can check this post](https://stackoverflow.com/questions/14366648/how-can-i-get-a-channel-id-from-youtube).   
Now imagine that you want to retrieve the IDs for a list of youtube channels, which can include one, two or several hundred youtube channels.  Then you should look for a better solution.   
An alternative solution is to use the YouTube API, if in the first place we need the id to use it, but we can work around to get the id of the channel, using the [`search`](https://developers.google.com/youtube/v3/guides/implementation/search) requests and the name of the channel. Drawback of this approach, it can return more than one result because the names of the channel are not necessarily unique. In addition, each search request costs 100 units, which can be an issue considering that the quota limit is 10,000 units per day ([quota usage](https://developers.google.com/youtube/v3/getting-started?hl=en#quota)).
 


In [1]:
from googleapiclient.discovery import build
import pandas as pd
import json
from dotenv import load_dotenv
import os
from tqdm import tqdm

## Load the data

First, we load a dataset of the youtube channels which we are looking for the IDs. The dataset was scraped from [Top Programmer Guru](https://noonies.tech/award/top-programming-guru) web page using the framework [`selenium`](https://github.com/SeleniumHQ/). The code is avaible [here](web_scraping.py).

In [2]:
channelList = pd.read_csv('data/top-programming-guru.csv')

In [3]:
channelList.head()

,channelName,url,rank
0,Programming with Mosh,https://www.youtube.com/c/programmingwithmosh/...,1
1,Traversy Media,https://www.youtube.com/user/TechGuyWeb,2
2,Corey Schafer,https://www.youtube.com/user/schafer5,3
3,Tech With Tim,https://m.youtube.com/channel/UC4JX40jDee_tINb...,4
4,Krish Naik,https://www.youtube.com/user/krishnaik06/playl...,5


In [4]:
channelList.shape

(71, 3)

### Clean the data

We going to check if theyre in any duplicate in `channelName` colomn then in the `url` colomn.

In [5]:
channelList['channelName'].str.lower().value_counts() #[channelList.channelName.value_counts()>1]

michael reeves              2
the coding train            2
maximilian schwarzmüller    1
chuck severance             1
naveen reddy                1
                           ..
thomas kim                  1
durga software solutions    1
fazt                        1
angelsix                    1
code basics                 1
Name: channelName, Length: 69, dtype: int64

We find two duplicate, we are going to remove

In [6]:
channelList.channelName[(channelList['channelName'].str.lower() == 'michael reeves') |
                        (channelList['channelName'].str.lower() == 'the coding train')]

14      Michael Reeves
16    The Coding Train
48    the coding train
68      Michael Reeves
Name: channelName, dtype: object

We can use [`drop_duplicates`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) method, but this going to  work only for one channel. Instead we are going to use the [`drop`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) method to remove the seconde duplicate.

In [7]:
channelList.drop(labels=[48, 68], inplace=True)

In [8]:
channelList.shape

(69, 3)

Let's check the `url` column.

In [9]:
channelList.url.value_counts().head()

https://www.youtube.com/c/Telusko                               3
https://www.youtube.com/c/HusseinNasser-software-engineering    2
https://www.youtube.com/dataschool                              1
https://m.youtube.com/results?search_query=code+with+harry      1
https://www.youtube.com/results?search_query=gaurav+sen         1
Name: url, dtype: int64

In [10]:
duplicate_url = channelList.url.value_counts()[channelList.url.value_counts()>1].index
duplicate_url

Index(['https://www.youtube.com/c/Telusko', 'https://www.youtube.com/c/HusseinNasser-software-engineering'], dtype='object')

In [11]:
duplicate_url

Index(['https://www.youtube.com/c/Telusko', 'https://www.youtube.com/c/HusseinNasser-software-engineering'], dtype='object')

In [12]:
channelList[channelList.url == duplicate_url[0]]

,channelName,url,rank
10,naveen Reddy,https://www.youtube.com/c/Telusko,11
27,Telusko,https://www.youtube.com/c/Telusko,28
44,Keith Galli,https://www.youtube.com/c/Telusko,45


We notice that three different channels have the same url, after a quick search on YouTube, we found that `naveeb Reddy` and `Telusko` should reference the same channel, the one run by `Navin Reddy`.  For [`Keith Galli`](https://www.youtube.com/c/KGMIT/featured) channel we will to look in YouTube for the correct url.

In [13]:
channelList.loc[44, 'url'] = 'https://www.youtube.com/c/KGMIT/featured'

In [14]:
channelList[channelList.url == duplicate_url[1]]

,channelName,url,rank
26,Ben Awad,https://www.youtube.com/c/HusseinNasser-softwa...,27
29,Hussein Nasser,https://www.youtube.com/c/HusseinNasser-softwa...,30


We will correct the url for `Ben Awad` channel.

In [15]:
channelList.loc[26, 'url'] = 'https://www.youtube.com/c/BenAwad97/featured'

After the correction of the wrong url, we will delete any remaining duplicates.

In [16]:
channelList.drop_duplicates(subset='url',ignore_index=True, inplace=True)

In [17]:
channelList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   channelName  68 non-null     object
 1   url          68 non-null     object
 2   rank         68 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.7+ KB


### Save the data
After removing the duplicate rows, we save the clean data in a new `csv` file.  

In [19]:
channelList.to_csv('data/channelListDB.csv', index=False)

In [20]:
# we store channels title in list
channels = channelList.channelName.values

## Load the API key

In [21]:
load_dotenv()
API_KEY = os.getenv('api_key2')
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [25]:
request = youtube.search().list(
        part="snippet",
        maxResults=1, # limit the search to one result
        q="Programming with Mosh",
        type="channel", # we going to look only for a youtube channels
    )
response = request.execute()


In [27]:
print(json.dumps(response,indent=4))

{
    "kind": "youtube#searchListResponse",
    "etag": "HacBt_2AqJV_Iu-Pp4VTDMJDkrE",
    "nextPageToken": "CAEQAA",
    "regionCode": "FR",
    "pageInfo": {
        "totalResults": 484,
        "resultsPerPage": 1
    },
    "items": [
        {
            "kind": "youtube#searchResult",
            "etag": "gX6q_SxI_mOFL3iiHS8UQX-pAA8",
            "id": {
                "kind": "youtube#channel",
                "channelId": "UCWv7vMbMWH4-V0ZXdmDpPBA"
            },
            "snippet": {
                "publishedAt": "2014-10-07T00:40:53Z",
                "channelId": "UCWv7vMbMWH4-V0ZXdmDpPBA",
                "title": "Programming with Mosh",
                "description": "I train professional software engineers that companies love to hire. My courses: http://codewithmosh.com My blog: http://programmingwithmosh.com Connect on ...",
                "thumbnails": {
                    "default": {
                        "url": "https://yt3.ggpht.com/ytc/AKedOLQ09D8wVCdgWo

In [28]:
response['items'][0]['snippet']['channelId']

'UCWv7vMbMWH4-V0ZXdmDpPBA'

### Display the result of the first request

In [29]:
for item in response['items']:
    print(item['snippet']['title'])
    print(item['snippet']['channelId'])
    print(item['id']['kind'])
    print('*' * 10)

Programming with Mosh
UCWv7vMbMWH4-V0ZXdmDpPBA
youtube#channel
**********


### Initialize dictionary to store the results

In [30]:
data = {'channelName': [], 'title': [], 'id':[], 'kind':[]}

### Add the first result to dictionary 

In [31]:
id = response['items'][0]['snippet']['channelId']
title = response['items'][0]['snippet']['title']
kind = response['items'][0]['id']['kind']

In [32]:
channel = channelList.loc[0, 'channelName']
channel

'Programming with Mosh'

In [33]:
data['channelName'].append(channel)
data['title'].append(title)
data['id'].append(id)
data['kind'].append(kind)

In [34]:
data

{'channelName': ['Programming with Mosh'],
 'title': ['Programming with Mosh'],
 'id': ['UCWv7vMbMWH4-V0ZXdmDpPBA'],
 'kind': ['youtube#channel']}

### Retrive the Id for each channel in dataset

In [36]:
for channel in tqdm(channels[1:]):

  request = youtube.search().list(
        part="snippet",
        maxResults=1,
        q=channel,
        type="channel"
    )
  
  response = request.execute()

  id = response['items'][0]['snippet']['channelId']
  title = response['items'][0]['snippet']['title']
  kind = response['items'][0]['id']['kind']

  data['channelName'].append(channel)
  data['title'].append(title)
  data['id'].append(id)
  data['kind'].append(kind)

100%|██████████| 67/67 [00:26<00:00,  2.48it/s]


In [37]:
data

{'channelName': ['Programming with Mosh',
  'Traversy Media',
  'Corey Schafer',
  'Tech With Tim',
  'Krish Naik',
  'freeCodeCamp.org',
  'Hitesh Choudhary',
  'Clever Programmer',
  'Caleb Curry',
  'programming Hero',
  'naveen Reddy',
  'Code Basics',
  'Maximilian Schwarzmüller',
  'sentdex',
  'Michael Reeves',
  'Dani',
  'The Coding Train',
  'Dennis Ivy',
  'CS Dojo',
  'Code with Harry',
  'Kalle Hallden',
  'Fireship',
  'Tim Corey',
  'TechLead',
  'WesBos',
  'Gajesh S. Naik',
  'Ben Awad',
  'Codevolution',
  'Hussein Nasser',
  'Dev Ed',
  'Brackeys',
  'Rachit Jain',
  'Kyle Cook',
  'Scott Hansellman',
  'PortEXE',
  'Joshua Fluke',
  'Web Dev Simplified',
  'Joma',
  'Tren Black',
  'Nick Chapsas',
  'CodingEntrepreneurs',
  'Programming With ERik',
  'Forest Knight',
  'Keith Galli',
  'Pretty Printed',
  'Derek Banas',
  'The Cherno',
  'Angelsix',
  'CodingWithMitch',
  'Chris Hawkes',
  'Chris Coyier',
  'Fazt',
  'Prajwal NH ',
  'Life of Luba',
  'chuck severan

In [38]:
df = pd.DataFrame.from_dict(data)

In [39]:
df.head()

,channelName,title,id,kind
0,Programming with Mosh,Programming with Mosh,UCWv7vMbMWH4-V0ZXdmDpPBA,youtube#channel
1,Traversy Media,Traversy Media,UC29ju8bIPH5as8OGnQzwJyA,youtube#channel
2,Corey Schafer,Corey Schafer,UCCezIgC97PvUuR4_gbFUs5g,youtube#channel
3,Tech With Tim,Tech With Tim,UC4JX40jDee_tINbkjycV4Sg,youtube#channel
4,Krish Naik,Krish Naik,UCNU_lfiiWBdtULKOw6X0Dig,youtube#channel


In [40]:
df['kind'].unique()

array(['youtube#channel'], dtype=object)

In [41]:
df.shape

(68, 4)

Let's check if there is a difference between the channel name and the title.

In [42]:
(df.channelName != df.title).sum()

22

Let's display the rows where the `channelName` and the `title` are different.

In [46]:
df[df.channelName != df.title]

,channelName,title,id,kind
9,programming Hero,Programming Hero,UCStj-ORBZ7TGK1FwtGAUgbQ,youtube#channel
10,naveen Reddy,Telusko,UC59K-uG2A5ogwIrHw4bmlEg,youtube#channel
11,Code Basics,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ,youtube#channel
12,Maximilian Schwarzmüller,Academind,UCSJbGtTlrDami-tDGPUV9-w,youtube#channel
19,Code with Harry,CodeWithHarry,UCeVMnSShP_Iviwkknt83cww,youtube#channel
22,Tim Corey,IAmTimCorey,UC-ptWR16ITQyYOglXyQmpzw,youtube#channel
24,WesBos,Wes Bos,UCoebwHSTvwalADTJhps0emA,youtube#channel
25,Gajesh S. Naik,Gajesh S Naik,UC7PWnwwqMSqAXQkKXqxRkMw,youtube#channel
33,Scott Hansellman,Scott Hanselman,UCL-fHOdarou-CR2XUmK48Og,youtube#channel
37,Joma,Joma Tech,UCV0qA-eDDICsRR9rPcnG7tw,youtube#channel


There are 22 rows where the value of the `channelName` column is different from the value of the `title` column, One of the reasons is that in the web page [Top Programmer Guru](https://noonies.tech/award/top-programming-guru)  the name of the channel owner is used as `channelName` instead of the official `title`, another reasons is case sensitivity.  
We are performing this verification because, as we mentioned at the beginning of the research method, it can return more than one result, so we need to make sure that we get the right channel ID.   
We check again the difference between the `channelName` column  and the `title` column but this time ignoring the case sensitivity.

In [47]:
(df.channelName.str.lower() != df.title.str.lower()).sum()

14

In [48]:
df[df.channelName.str.lower() != df.title.str.lower()]

,channelName,title,id,kind
10,naveen Reddy,Telusko,UC59K-uG2A5ogwIrHw4bmlEg,youtube#channel
11,Code Basics,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ,youtube#channel
12,Maximilian Schwarzmüller,Academind,UCSJbGtTlrDami-tDGPUV9-w,youtube#channel
19,Code with Harry,CodeWithHarry,UCeVMnSShP_Iviwkknt83cww,youtube#channel
22,Tim Corey,IAmTimCorey,UC-ptWR16ITQyYOglXyQmpzw,youtube#channel
24,WesBos,Wes Bos,UCoebwHSTvwalADTJhps0emA,youtube#channel
25,Gajesh S. Naik,Gajesh S Naik,UC7PWnwwqMSqAXQkKXqxRkMw,youtube#channel
33,Scott Hansellman,Scott Hanselman,UCL-fHOdarou-CR2XUmK48Og,youtube#channel
37,Joma,Joma Tech,UCV0qA-eDDICsRR9rPcnG7tw,youtube#channel
41,Programming With ERik,Program With Erik,UCshZ3rdoCLjDYuTR_RBubzw,youtube#channel


Now we only have `13` rows left to verify. Quick check in Youtube and we see that there is only one error, it is in row 52. Let's correct the error and save the data.

In [49]:
channle_52 = {'title': 'IIMB Inventors Inventing Machine Business', 'id':'UCWym_j-OGCzIfz8K4vXtU6g'}

In [50]:
df.loc[52, 'title'] = 'IIMB Inventors Inventing Machine Business'
df.loc[52, 'id'] = 'UCWym_j-OGCzIfz8K4vXtU6g'

In [51]:
df.loc[52]

channelName                                  Prajwal NH 
title          IIMB Inventors Inventing Machine Business
id                              UCWym_j-OGCzIfz8K4vXtU6g
kind                                     youtube#channel
Name: 52, dtype: object

We going to store data in `csv` file

In [52]:
df.to_csv('data/channelsID.csv', index=False)

In [9]:
channelsID = pd.read_csv('data/channelsID.csv')
channelList = pd.read_csv('data/channelListDB.csv')

In [10]:
channelsID.shape

(68, 4)

In [11]:
channelList.shape

(68, 3)

When we merge the two datasets above to form a single dataset.  

In [12]:
df = pd.merge(channelsID, channelList, on='channelName')
df.head(10)

,channelName,title,id,kind,url,rank
0,Programming with Mosh,Programming with Mosh,UCWv7vMbMWH4-V0ZXdmDpPBA,youtube#channel,https://www.youtube.com/c/programmingwithmosh/...,1
1,Traversy Media,Traversy Media,UC29ju8bIPH5as8OGnQzwJyA,youtube#channel,https://www.youtube.com/user/TechGuyWeb,2
2,Corey Schafer,Corey Schafer,UCCezIgC97PvUuR4_gbFUs5g,youtube#channel,https://www.youtube.com/user/schafer5,3
3,Tech With Tim,Tech With Tim,UC4JX40jDee_tINbkjycV4Sg,youtube#channel,https://m.youtube.com/channel/UC4JX40jDee_tINb...,4
4,Krish Naik,Krish Naik,UCNU_lfiiWBdtULKOw6X0Dig,youtube#channel,https://www.youtube.com/user/krishnaik06/playl...,5
5,freeCodeCamp.org,freeCodeCamp.org,UC8butISFwT-Wl7EV0hUK0BQ,youtube#channel,https://www.youtube.com/channel/UC8butISFwT-Wl...,6
6,Hitesh Choudhary,Hitesh Choudhary,UCXgGY0wkgOzynnHvSEVmE3A,youtube#channel,https://www.youtube.com/c/HiteshChoudharydotcom,7
7,Clever Programmer,Clever Programmer,UCqrILQNl5Ed9Dz6CGMyvMTQ,youtube#channel,https://m.youtube.com/cleverprogrammer?uid=qrI...,8
8,Caleb Curry,Caleb Curry,UCZUyPT9DkJWmS_DzdOi7RIA,youtube#channel,https://www.youtube.com/user/CalebTheVideoMaker2,9
9,programming Hero,Programming Hero,UCStj-ORBZ7TGK1FwtGAUgbQ,youtube#channel,https://www.youtube.com/channel/UCStj-ORBZ7TGK...,10


In [13]:
df.rename({'id':'channelId'}, inplace=True, axis=1)
df.head()

,channelName,title,channelId,kind,url,rank
0,Programming with Mosh,Programming with Mosh,UCWv7vMbMWH4-V0ZXdmDpPBA,youtube#channel,https://www.youtube.com/c/programmingwithmosh/...,1
1,Traversy Media,Traversy Media,UC29ju8bIPH5as8OGnQzwJyA,youtube#channel,https://www.youtube.com/user/TechGuyWeb,2
2,Corey Schafer,Corey Schafer,UCCezIgC97PvUuR4_gbFUs5g,youtube#channel,https://www.youtube.com/user/schafer5,3
3,Tech With Tim,Tech With Tim,UC4JX40jDee_tINbkjycV4Sg,youtube#channel,https://m.youtube.com/channel/UC4JX40jDee_tINb...,4
4,Krish Naik,Krish Naik,UCNU_lfiiWBdtULKOw6X0Dig,youtube#channel,https://www.youtube.com/user/krishnaik06/playl...,5


In [14]:
df.to_csv('data/channelsMergeDB.csv', index=False)

Before contiune in the project,  we  will to verife many each and new column. save the final dataset in `channelsList.csv`

In [15]:
channelList = pd.read_csv('data/channelsList.csv')

In [16]:
channelList.head()

,channelName,title,channelId,kind,url,gender,rank
0,Programming with Mosh,Programming with Mosh,UCWv7vMbMWH4-V0ZXdmDpPBA,youtube#channel,https://www.youtube.com/c/programmingwithmosh/...,Male,1
1,Traversy Media,Traversy Media,UC29ju8bIPH5as8OGnQzwJyA,youtube#channel,https://www.youtube.com/user/TechGuyWeb,Male,2
2,Corey Schafer,Corey Schafer,UCCezIgC97PvUuR4_gbFUs5g,youtube#channel,https://www.youtube.com/user/schafer5,Male,3
3,Tech With Tim,Tech With Tim,UC4JX40jDee_tINbkjycV4Sg,youtube#channel,https://m.youtube.com/channel/UC4JX40jDee_tINb...,Male,4
4,Krish Naik,Krish Naik,UCNU_lfiiWBdtULKOw6X0Dig,youtube#channel,https://www.youtube.com/user/krishnaik06/playl...,Male,5
